<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/Astra_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
import itertools

# Загрузка данных
data = pd.read_excel('Data_Set4.xlsx')

# Очистка данных
data.replace('н.д.', np.nan, inplace=True)
data['Возраст'].fillna(data['Возраст'].median(), inplace=True)

# Очистка строковых данных от пробелов
for col in ['Пол', 'Роль в суде', 'Рецидив', 'Был ли осуществлен привод']:
    data[col] = data[col].str.strip()

# Инициализация LabelEncoder для каждого категориального столбца и его обучение
le_dict = {}
categorical_columns = ['Пол', 'Роль в суде', 'Рецидив']
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    le_dict[column] = le

# Кодирование целевой переменной
le_target = LabelEncoder()
data['Был ли осуществлен привод'] = le_target.fit_transform(data['Был ли осуществлен привод'])
print("Уникальные значения целевой переменной после кодирования:", le_target.classes_)

# Целевая переменная и признаки
X = data[['Пол', 'Возраст', 'Роль в суде', 'Рецидив']]
y = data['Был ли осуществлен привод']

# Проверка распределения классов до балансировки
print("Распределение классов до SMOTE:")
print(y.value_counts())

# Балансировка классов с помощью SMOTE (многоклассовая)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Проверка распределения классов после SMOTE
print("Распределение классов после SMOTE:")
print(pd.Series(y_resampled).value_counts())

# Разделение данных на обучающую и тестовую выборки с стратификацией
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42,
    stratify=y_resampled
)

# Проверка распределения классов в обучающей выборке
print("Распределение классов в обучающей выборке (y_train):")
print(pd.Series(y_train).value_counts())

# Проверка распределения классов в тестовой выборке
print("Распределение классов в тестовой выборке (y_test):")
print(pd.Series(y_test).value_counts())

# Настройка гиперпараметров с помощью GridSearchCV для XGBoost
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}

model = XGBClassifier(random_state=42, eval_metric='mlogloss')  # Удален use_label_encoder
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print(f"Лучшие параметры: {grid_search.best_params_}")

# Оценка модели
predictions = best_model.predict(X_test)
predictions_proba = best_model.predict_proba(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Точность модели: {accuracy:.2f}")

print("Отчет о классификации:")
print(classification_report(y_test, predictions, zero_division=0))

# Вычисление ROC-AUC для многоклассовой классификации
roc_auc = roc_auc_score(y_test, predictions_proba, multi_class='ovr', average='macro')
print(f"ROC-AUC (OVR, Macro): {roc_auc:.2f}")

# Генерация предсказаний для новых данных и сохранение в файл ASTRA.xlsx
# Определение возможных значений признаков
genders = le_dict['Пол'].transform(['Мужской', 'Женский'])
roles = le_dict['Роль в суде'].transform(['Обвиняемый', 'Потерпевший', 'Свидетель', 'Должник'])
recidivisms = le_dict['Рецидив'].transform(['Да', 'Нет'])
age_range = range(16, 81)  # Возраст от 16 до 80 лет

# Создание списка всех комбинаций признаков
combinations = list(itertools.product(genders, age_range, roles, recidivisms))

# Создание DataFrame с комбинациями
new_data = pd.DataFrame(combinations, columns=['Пол', 'Возраст', 'Роль в суде', 'Рецидив'])

# Предсказание вероятностей
probabilities = best_model.predict_proba(new_data)

# Добавление вероятностей в DataFrame
for idx, class_label in enumerate(le_target.classes_):
    new_data[f'Вероятность {class_label}'] = probabilities[:, idx]

# Обратное преобразование кодированных признаков в исходные категории
for column in categorical_columns:
    le = le_dict[column]
    new_data[column] = le.inverse_transform(new_data[column])

# Обратное преобразование целевой переменной, если необходимо
# Например, можно определить вероятность для каждого класса

# Сохранение результатов в файл ASTRA.xlsx
new_data.to_excel('ASTRA.xlsx', index=False)
print("Результаты сохранены в файл ASTRA.xlsx")

Уникальные значения целевой переменной после кодирования: ['Да' 'Находился под заключением' 'Нет']
Распределение классов до SMOTE:
Был ли осуществлен привод
0    2520
2    2320
1     400
Name: count, dtype: int64
Распределение классов после SMOTE:
Был ли осуществлен привод
0    2520
2    2520
1    2520
Name: count, dtype: int64
Распределение классов в обучающей выборке (y_train):
Был ли осуществлен привод
2    2016
1    2016
0    2016
Name: count, dtype: int64
Распределение классов в тестовой выборке (y_test):
Был ли осуществлен привод
2    504
1    504
0    504
Name: count, dtype: int64
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
Точность модели: 0.85
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.90      0.70      0.79       504
           1       0.84      1.00      0.91       504
           2       0.81      0.84      0.83       504

    accuracy                           0.85   